In [2]:
from functools import partial

from astropy.coordinates import SkyCoord
from astropy.time import Time
import astropy.units as u
from astroquery.simbad import Simbad
import pandas as pd

In [3]:
catalog_name = "Gaia DR2 RV Standard Stars"
gaia_table = pd.read_fwf(f"../../Files/Spectroscopic Standards/{catalog_name}/rvstdcat.dat")
gaia_table.columns = [
    "ID",
    "RV",
    "eRV",
    "s_RV",
    "e_RV",
    "Tbase",
    "JDm",
    "N",
    "Source",
    "Flag",
    "RVS",
    "e_RVS",
    "o_RVS",
    "2MASS",
    "Jmag",
    "Hmag",
    "Kmag",
    "e_Jmag",
    "e_Hmag",
    "e_Kmag",
    "Qflag",
    "RAdeg",
    "DEdeg",
    "Epoch",
    "Bmag",
    "Vmag",
    "SpType",
    "otype",
]

gaia_table = gaia_table[
    ~(gaia_table["SpType"] == "~")
    & (gaia_table["otype"] == "Star")
    & (gaia_table["Vmag"] < 11)
    & (gaia_table["e_RV"] < 0.02)
    & (gaia_table["ID"].str.contains("HIP"))
]
gaia_table["ID"] = [f"HIP {int(id)}" for id in gaia_table["ID"].str.replace("HIP", "")]

gaia_table

,ID,RV,eRV,s_RV,e_RV,Tbase,JDm,N,Source,Flag,...,e_Hmag,e_Kmag,Qflag,RAdeg,DEdeg,Epoch,Bmag,Vmag,SpType,otype
140,HIP 142,-21.721,0.0015,0.0192,0.0035,1473,54743,31,432340826884417792,CAL1,...,0.038,0.018,AAA,0.453851,66.30608,J2015.5,8.05,7.32,K0,Star
141,HIP 184,-17.512,0.0020,0.0092,0.0041,4172,54549,5,2765462891378117504,CAL1,...,0.036,0.033,AAA,0.589740,11.00605,J2015.5,9.33,8.47,K0V,Star
163,HIP 669,8.044,0.0003,0.0178,0.0043,3685,54443,17,2417456431710276096,CAL1,...,0.042,0.034,AAA,2.068110,-14.82454,J2015.5,7.65,7.03,G1V,Star
171,HIP 867,32.090,0.0020,0.0300,0.0100,4026,54387,9,2767585258056231040,CAL1,...,0.016,0.023,AAA,2.663211,12.82016,J2015.5,9.03,8.46,G5,Star
172,HIP 870,17.280,0.0025,0.0107,0.0054,3274,55295,4,2546769822454212992,CAL1,...,0.021,0.026,AAA,2.669626,2.05563,J2015.5,7.97,7.51,F(6)V,Star
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4469,HIP 116557,98.660,0.0170,0.0167,0.0098,1103,55391,3,6489233278272505216,CAL1,...,0.018,0.027,AAA,354.318726,-59.49041,J2015.5,10.19,9.36,G6/8IV,Star
4474,HIP 116743,-31.885,0.0016,0.0205,0.0037,3288,54795,30,2011056971082466944,CAL1,...,0.033,0.018,AAA,354.895874,58.97053,J2015.5,8.38,7.52,K0V,Star
4494,HIP 117390,-55.584,0.0025,0.0060,0.0030,2459,55168,4,1921294044375808896,VAL,...,0.055,0.026,AAA,357.035187,41.10787,J2015.5,8.93,8.27,G5,Star
4508,HIP 117733,14.841,0.0030,0.0042,0.0024,2573,55219,3,1993638920193452160,VAL,...,0.016,0.016,AAA,358.165253,54.26866,J2015.5,8.07,7.58,F5,Star


In [4]:
simbad = Simbad()
simbad.TIMEOUT = 120
simbad.add_votable_fields("ids")
simbad_result = simbad.query_objects(gaia_table["ID"]).to_pandas()
len(simbad_result)

382

In [5]:
def extract_id(id_list, id_name):
    for id in id_list.split("|"):
        if id_name in id:
            return id.replace(id_name, "").strip()
    return ""


simbad_result["TIC"] = simbad_result["IDS"].apply(partial(extract_id, id_name="TIC"))
simbad_result["HIP"] = simbad_result["IDS"].apply(partial(extract_id, id_name="HIP"))
simbad_result = simbad_result[simbad_result["TIC"] > ""].reset_index()

gaia_ids = set(int(id) for id in gaia_table["ID"].str.replace("HIP ", ""))
simbad_ids = set([int(id) for id in simbad_result["HIP"]])
no_simbad =  gaia_ids - simbad_ids

gaia_table = gaia_table[~gaia_table["ID"].isin([f"HIP {id}" for id in no_simbad])]

len(no_simbad), len(gaia_table)

(4, 378)

In [24]:
gaia_times = Time(gaia_table["Epoch"].to_list(), format="jyear_str")
gaia_coords = SkyCoord(ra=gaia_table["RAdeg"], dec=gaia_table["DEdeg"], obstime=gaia_times, unit=(u.deg, u.deg))

simbad_coords = SkyCoord(ra=simbad_result["RA"], dec=simbad_result["DEC"], unit=(u.hourangle, u.deg))

dist = gaia_coords.separation(simbad_coords)
(dist > 1 * u.deg).any()

False

In [28]:

targets_file = pd.DataFrame()
targets_file["Target Name"] =gaia_table["ID"]
targets_file["RA"] = gaia_coords.ra.to_string(unit=u.hour, decimal=False, sep=":")
targets_file["DEC"] = gaia_coords.dec.to_string(unit=u.deg, decimal=False, sep=":", alwayssign=True)
targets_file["Target Type"] = "RV Standard"
targets_file["Target Source"] = f"{catalog_name}, doi.org/10.1051/0004-6361/201832795"
targets_file.to_csv(f"../../Data/Targets/{catalog_name}.csv", index=False)

targets_file

,Target Name,RA,DEC,Target Type,Target Source
140,HIP 142,0:01:48.92424,+66:18:21.888,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00..."
141,HIP 184,0:02:21.5376,+11:00:21.78,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00..."
171,HIP 867,0:10:39.17064,+12:49:12.576,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00..."
172,HIP 870,0:10:40.71024,+2:03:20.268,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00..."
176,HIP 920,0:11:24.38832,+23:49:06.096,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00..."
...,...,...,...,...,...
4469,HIP 116557,23:37:16.49424,-59:29:25.476,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00..."
4474,HIP 116743,23:39:35.00976,+58:58:13.908,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00..."
4494,HIP 117390,23:48:08.44488,+41:06:28.332,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00..."
4508,HIP 117733,23:52:39.66072,+54:16:07.176,RV Standard,"Gaia DR2 RV Standard Stars, doi.org/10.1051/00..."


In [26]:
simbad_result

,index,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,IDS,SCRIPT_NUMBER_ID,TIC,HIP
0,0,HD 224918,00 01 48.9086,+66 18 21.717,14,14,0.0116,0.0133,90,A,O,2020yCat.1350....0G,HD 224918|AG+66 1245|AGKR 21487|BD+65 1984|GC...,1,378534800,142
1,1,HD 224983,00 02 21.5428,+11 00 22.455,14,14,0.0190,0.0135,90,A,O,2020yCat.1350....0G,HD 224983|** KPP 4405A|AG+10 3288|AGKR 21495|B...,2,403021108,184
2,3,HD 631,00 10 39.1283,+12 49 12.835,14,14,0.0264,0.0237,90,A,O,2020yCat.1350....0G,HD 631|AG+12 19|AGKR 111|BD+11 10|FK...,4,357395017,867
3,4,HD 633,00 10 40.7234,+02 03 19.959,14,14,0.0249,0.0192,90,A,O,2020yCat.1350....0G,HD 633|AG+01 11|AGKR 113|BD+01 12|GE...,5,427330376,870
4,5,HD 700,00 11 24.4280,+23 49 05.610,14,14,0.0197,0.0169,90,A,O,2020yCat.1350....0G,HD 700|AG+23 18|BD+23 10|GEN# +1.00000...,6,150784941,920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,377,HD 221998,23 37 16.5392,-59 29 25.337,14,14,0.0094,0.0098,90,A,O,2020yCat.1350....0G,HD 221998|CD-60 8091|CPC 20 7511|CPD-60 766...,378,219336263,116557
374,378,HD 222366,23 39 34.9891,+58 58 13.662,14,14,0.0132,0.0119,90,A,O,2020yCat.1350....0G,HD 222366|AG+58 1575|BD+58 2627|BSD 19-258|G...,379,344852533,116743
375,379,HD 223276,23 48 08.4152,+41 06 28.641,14,14,0.0173,0.0139,90,A,O,2020yCat.1350....0G,HD 223276|AG+40 2493|BD+40 5150|CCDM J23481+4...,380,396194157,117390
376,380,HD 223788,23 52 39.6679,+54 16 07.788,14,14,0.0119,0.0122,90,A,O,2020yCat.1350....0G,HD 223788|** SHY 840C|AG+53 1692|BD+53 3244|...,381,65275623,117733


In [136]:
catalog_members = pd.DataFrame()
catalog_members["Target Name"] = gaia_table["ID"]
catalog_members["Catalog Name"] = "TESS TICv8"
catalog_members["Catalog ID"] = [str(id) for id in simbad_result["TIC"]]
catalog_members.to_csv(f"../../Data/Catalog Members/{catalog_name}.csv", index=False)

catalog_members

,Target Name,Catalog Name,Catalog ID
140,HIP 142,TESS TICv8,378534800
141,HIP 184,TESS TICv8,403021108
171,HIP 867,TESS TICv8,357395017
172,HIP 870,TESS TICv8,427330376
176,HIP 920,TESS TICv8,150784941
...,...,...,...
4469,HIP 116557,TESS TICv8,219336263
4474,HIP 116743,TESS TICv8,344852533
4494,HIP 117390,TESS TICv8,396194157
4508,HIP 117733,TESS TICv8,65275623


In [128]:
rv_standards = pd.DataFrame()
rv_standards["Target Name"] = gaia_table["ID"]
rv_standards["Spectral Type"] = gaia_table["SpType"]
rv_standards["RV"] = gaia_table["RV"]
rv_standards["RV err"] = gaia_table["e_RV"]
rv_standards.to_csv(f"../../Data/RV Calibration Targets/{catalog_name}.csv", index=False)

rv_standards

,Target Name,Spectral Type,RV,RV err
140,HIP 142,K0,-21.721,0.0035
141,HIP 184,K0V,-17.512,0.0041
171,HIP 867,G5,32.090,0.0100
172,HIP 870,F(6)V,17.280,0.0054
176,HIP 920,G0V,4.314,0.0082
...,...,...,...,...
4469,HIP 116557,G6/8IV,98.660,0.0098
4474,HIP 116743,K0V,-31.885,0.0037
4494,HIP 117390,G5,-55.584,0.0030
4508,HIP 117733,F5,14.841,0.0024
